In [1]:
from Dataset_v1 import Dataset_v1
from Model_v1 import Model_v1
from Analysis import FeatureAnalysis 
from ModelEval import ModelEval
from walk_forward import walk_forward,wf_get_topk_acc
import pandas as pd
import numpy as np

In [6]:
### WALK FORWARD ###
wf = walk_forward(["Qual_Position","driver_encoding"])

[(2025, 1), (2025, 2), (2025, 3), (2025, 4), (2025, 5), (2025, 6), (2025, 7), (2025, 8)]


Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 2) (236,)
(153, 2) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 2) (1221,)
(236, 

In [7]:
#wf[(wf["Round_Number"]==7)]
wf[(wf["Location"]=="Shanghai")]

,Qual_Position,driver_encoding,preds,probs,actual,Location,Year,Round_Number,BroadcastName
16,2.0,5.392158,1,0.961651,1,Shanghai,2025,2,G RUSSELL
17,1.0,8.264109,1,0.935984,1,Shanghai,2025,2,O PIASTRI
18,3.0,5.073370,1,0.911740,1,Shanghai,2025,2,L NORRIS
19,4.0,4.909265,1,0.907947,0,Shanghai,2025,2,M VERSTAPPEN
20,6.0,5.924366,1,0.634495,0,Shanghai,2025,2,C LECLERC
21,5.0,8.496419,1,0.627419,0,Shanghai,2025,2,L HAMILTON
22,15.0,9.313935,0,0.305637,0,Shanghai,2025,2,C SAINZ
23,14.0,12.125813,0,0.063924,0,Shanghai,2025,2,L STROLL
24,20.0,12.031016,0,0.037935,0,Shanghai,2025,2,L LAWSON
25,12.0,10.200452,0,0.021339,0,Shanghai,2025,2,N HULKENBERG


In [8]:
wf_get_topk_acc(wf)

Avg TopK Acc:0.9166666666666666


,Location,Year,Acc,n_actual
0,Imola,2025,1.000000,3
1,Jeddah,2025,1.000000,3
2,Melbourne,2025,1.000000,3
3,Miami,2025,0.666667,3
4,Monaco,2025,1.000000,3
5,Sakhir,2025,0.666667,3
6,Shanghai,2025,1.000000,3
7,Suzuka,2025,1.000000,3


In [5]:
wf = walk_forward(["Qual_Position","driver_encoding","n_past_podiums_last_5"])
wf_get_topk_acc(wf)

[(2025, 1), (2025, 2), (2025, 3), (2025, 4), (2025, 5), (2025, 6), (2025, 7), (2025, 8)]


Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)



Features for training set:
 ['Qual_Position', 'driver_encoding', 'n_past_podiums_last_5']

Dataset Shapes
(1221, 3

,Location,Year,Acc,n_actual
0,Imola,2025,1.000000,3
1,Jeddah,2025,0.666667,3
2,Melbourne,2025,0.666667,3
3,Miami,2025,0.666667,3
4,Monaco,2025,1.000000,3
5,Sakhir,2025,0.666667,3
6,Shanghai,2025,1.000000,3
7,Suzuka,2025,1.000000,3


In [ ]:
train_data = pd.read_csv("data/train_data_new.csv")
test_data = pd.read_csv("data/test_data_new.csv")
data_base = pd.concat([train_data,test_data])

In [ ]:
dataset = Dataset_v1("data/train_data_new.csv","data/test_data_new.csv",False)
dataset.build_features_into_dataset()
analysis = FeatureAnalysis(dataset)
data=dataset.get_data()

In [ ]:
#data.loc[(data["Race_Time_Encoding"]==1)&(data["target"]==1),["Location","Round_Number","Year","BroadcastName","Qual_Q3_Time_Normal","Qual_Position","Race_Time_Encoding","Race_Position"]]

In [ ]:
def check_standardized_time_ordering(df):
    def is_time_ascending(group):
        # Sort by Race_Position
        group_sorted = group.sort_values("Qual_Position", ascending=True)
        group_sorted.replace(-1,1000,inplace=True)
        # Check if Standardized_Time is non-decreasing
        valid_times = group_sorted["Qual_Q3_Time"].dropna()
        return valid_times.is_monotonic_increasing

    # Apply the check to each (Year, Round_Number) group
    result = (
        df.groupby(["Year", "Round_Number"])
        .apply(is_time_ascending)
        .reset_index(name="is_time_ordered")
    )

    return result

In [ ]:
res = check_standardized_time_ordering(data_base)
res.loc[res["is_time_ordered"]==False]

In [ ]:
row_condition = (data_base["Year"]==2025)&(data_base["Round_Number"]==7) 
data_base.loc[row_condition,["Location","TeamName","BroadcastName","Qual_Q3_Time","Qual_Position","Standardized_Time","Race_Position"]].head(20).sort_values(by="Race_Position")

In [ ]:

def yuh(year,round):
    row_condition = (data["Year"]==year)&(data["Round_Number"]==round)
    info = data.loc[row_condition,["Location","Year","Round_Number","BroadcastName","n_past","Qual_Position","driver_encoding","Race_Position"]].sort_values(by="Qual_Position")
    print(len(info))
    return info 

In [ ]:
#data[(data['BroadcastName'] == 'S PEREZ') & (data['Year'] == 2024)][['Race_Date_Code', 'Race_Position', 'n_past', 'driver_encoding']]


In [ ]:
yuh(2022,15) 

In [ ]:
### MODEL EVAL EXPERIMENTING ###

dataset = Dataset_v1("data/train_data_new.csv","data/test_data_new.csv",False)
dataset.build_features_into_dataset()

features_for_training = [
    "n_past_podiums_last_5",
    "Qual_Position",
    "pos_gained_encoding_last_5",
    #"ewa_driver_results",
    "driver_encoding",
    #"Race_Time_Encoding",
    #"Qual_Q3_Time_Normal",
    #"pos_gained_encoding_simple",
    #"n_past_podiums",
    #"pos_gained_encoding",
    #"TopTeam_Red Bull Racing",
    #"TopTeam_Ferrari",
    #"TopTeam_McLaren",
    #"Sprint_Race_Position",
    #"Sprint_Qual_Position",
    #"TopTeam_Mercedes"
]

dataset.set_features_for_training(features_for_training)

model = Model_v1(dataset,"RF_trn",False)
model.train()
print(model.get_feature_importance())
model_eval = ModelEval(model,dataset)
model_eval_df = model_eval.get_df()

In [ ]:
model_eval.get_preds_for_race(2025,6)

In [ ]:
model_eval.get_driver_with_most_wrong_predictions()

In [ ]:
gr = model_eval.get_driver_with_most_wrong_predictions()
gr.loc[gr["Year"]==2025,:]

In [ ]:
model_eval.get_drivers_with_most_predictions()

In [ ]:
#analysis.plot_prob_dist("Race_Time_Encoding")

In [ ]:
analysis.plot_prob_dist("driver_encoding")

In [ ]:
#analysis.plot_prob_dist("lag_SpeedST")

In [ ]:
#analysis.plot_prob_dist("ewa_driver_results")

In [ ]:
def create_pos_gained_encoding(driver, qual_position, race_position):
    """
    # Convert inputs to lists if they're pandas Series
    if isinstance(driver, pd.Series):
        driver = driver.tolist()
    if isinstance(qual_position, pd.Series):
        qual_position = qual_position.tolist()
    if isinstance(race_position, pd.Series):
        race_position = race_position.tolist()
    """
    
    # Handle -1 in race_position
    race_position = [21 if x == -1 else x for x in race_position]
    
    # Calculate position gained
    pos_gained = [q - r for q, r in zip(qual_position, race_position)]

    def calc(x): return (1/np.log(x+1))*((21-x)/20)
    
    # Calculate finishing weights
    finishing_weight = [calc(x) for x in race_position]
    
    final = [] 
    podium_bonus = 1

    for i in range(len(pos_gained)):
        if pos_gained[i] == 0:
            adjusted_pos_gained = finishing_weight[i]
        elif pos_gained[i] > 0:
            adjusted_pos_gained = pos_gained[i]*finishing_weight[i]
        else:
            if finishing_weight[i] <= 0.0: 
                adjusted_pos_gained = pos_gained[i]*(-np.log(finishing_weight[i]+0.000001))*0.05
            else:
                adjusted_pos_gained = pos_gained[i]*(-np.log(finishing_weight[i]))*0.05
            
        if race_position[i] <= 3:
            final.append(adjusted_pos_gained + podium_bonus)
        else:
            final.append(adjusted_pos_gained)

    df = pd.DataFrame({
        "driver": driver,
        "qual_position": qual_position,
        "race_pos": race_position,
        "pos_gained": pos_gained,
        "finishing_weight": finishing_weight,
        "final": final
    }).sort_values(by="final", ascending=False)
    
    return df

In [ ]:
race_results = data.loc[:, ['BroadcastName','Qual_Position','Race_Position']].iloc[20:40]
create_pos_gained_encoding(race_results["BroadcastName"],race_results["Qual_Position"],race_results["Race_Position"])

In [ ]:
race_results = data.loc[:, ['BroadcastName','Qual_Position','Race_Position']].iloc[0:20]
create_pos_gained_encoding(race_results["BroadcastName"],race_results["Qual_Position"],race_results["Race_Position"])